# CSprites Config Generator

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import pickle as pkl
import timeit
#
import numpy as np
import pprint
#
from csprites.masks import get_shapes
from csprites.scales import get_shape_sizes_evenly_scaled
from csprites.positions import get_max_positions
from csprites.configs import be_config

# Backend Parameters

In [ ]:
pprint.pprint(be_config)

# User Input: Generative Factors

In [ ]:
be_config.all_shape_names

In [ ]:
# Independet User Inputs
# see be_config for ranges and limits, check client side!
# img size
img_size = 64
assert img_size in be_config.img_sizes

# shapes
shape_names = be_config.all_shape_names
for name in shape_names:
    assert name in be_config.all_shape_names

# fill rates
min_mask_fill_rate = 0.2
assert 0 < min_mask_fill_rate < 1

max_mask_fill_rate = 0.5  
assert min_mask_fill_rate <= max_mask_fill_rate
assert max_mask_fill_rate < 1

# colors 
n_colors = 512
assert be_config.n_colors_min <= n_colors
assert n_colors <= be_config.n_colors_max

# angles
n_angles = 32
assert be_config.n_angles_min <= n_angles
assert n_angles <= be_config.n_angles_max

# backgrounds
n_bg = np.inf                                 # n_bgs_min <= n_bg <= n_bgs_max [1, n, np.inf]
if n_bg != np.inf:
    assert be_config.n_bgs_min <= n_bg
    assert n_bg <= be_config.n_bgs_max

bg_style = "random_function"              # bg_style in bg_styles
assert bg_style in be_config.bg_styles

# targets
target_bbox = True
target_segm = True

# Intermediate Calculations

In [ ]:
# Shapes
shapes = get_shapes(shape_names)
n_shapes = len(shapes)

# shape_sizes
min_mask_area = img_size** 2 * min_mask_fill_rate
max_mask_area = img_size** 2 * max_mask_fill_rate

# Notes: If limited to 0.1 steps or so, we could precalculate the max_mask_size and subsequently max positions ect
shape_sizes, max_mask_size = get_shape_sizes_evenly_scaled(shapes, min_mask_area, max_mask_area)

# n_scales_max
n_scales_max = len(list(shape_sizes.values())[0])

# n_positions_max
n_positions_max = get_max_positions(img_size, max_mask_size)

# User Input: Generative Factors, that depend on Previous User Input

In [ ]:
print(n_scales_max)
print(n_positions_max)

In [ ]:
# Dependent Inputs
n_scales = n_scales_max        # n_scales_min <= n_scales <= n_scales_max
assert be_config.n_scales_min <= n_scales
assert n_scales <= be_config.n_scales_max
assert n_scales <= n_scales_max

n_positions = 10  # n_positions_min <= n_positions <= n_positions_max
assert be_config.n_positions_min <= n_positions
assert n_positions <= be_config.n_positions_max
assert n_positions <= n_positions_max

# Visualize Inputs

In [ ]:
print("img dims:   {}x{}x{}".format(img_size, img_size, 3))
print("#shapes:    {}".format(n_shapes))
print("#colors:    {}".format(n_colors))
print("#angles:    {}".format(n_angles))
print("#scales:    {}".format(n_scales))
print("#positions: {}".format(n_positions))

# User Input: Dataset Stats

In [ ]:
# we do not care about backgrounds!
# some masks are the same, thanks to rotation invariance
n_masks = n_shapes * n_colors * n_angles * n_scales
n_states = n_masks * n_positions**2
#
mem_usage_bytes = img_size**2 * 3 * n_states
#
print("*"*40)
print("{:^40}".format("Space Stats"))
print("*"*40)
print("#States:        {:<10}".format(n_states))
print("#Masks:         {:<10}".format(n_masks))
print("Mem usage [mb]: {:.2f}".format(mem_usage_bytes * 1e-6))
print("Mem usage [Gb]: {:.2f}".format(mem_usage_bytes * 1e-9))# Calculate Stuff

In [ ]:
###### INPUT #####
generate_subset = True  # Generate subset or whole state space?
##################

if generate_subset:
    ###### INPUT #####
    n_train = 90000
    n_valid = 10000
    ##################
    n_samples = n_train + n_valid
    train_rate = n_train / n_samples
    #
    assert n_samples <= n_states, "Cannot generate more than all possible states!"
else:
    n_samples = n_states
    ###### INPUT #####
    train_rate = 0.9
    ##################generate_subset
    n_train = int(train_rate * n_states)
    n_valid = n_states - n_train

samling_rate = n_samples / n_states
mem_usage_bytes = img_size**2 * 3 * n_samples
    
assert be_config.mem_usage_gb_max >= mem_usage_bytes * 1e-9

In [ ]:
print("*"*40)
print("{:^40}".format("Dataset Stats"))
print("*"*40)
print("#Masks:        {:>10}".format(n_masks))
print("#States:       {:>10}".format(n_states))
print("#Samples:      {:>10}".format(n_samples))
print("#Train:        {:>10}".format(n_train))
print("#Valid:        {:>10}".format(n_valid))
print("*"*40)
print("Train rate:     {:.2f}".format(train_rate))
print("Sampling rate:  {:.2f}".format(samling_rate))
print("Mem usage [mb]: {:.2f}".format(mem_usage_bytes * 1e-6))
print("Mem usage [Gb]: {:.2f}".format(mem_usage_bytes * 1e-9))# Calculate Stuff

# Generate Config File
Used to generate the dataset

In [ ]:
# create copy of template dict
config = {}
config['n_bg'] = n_bg
config['bg_style'] = bg_style
config['shapes'] = shape_names
config["n_scales"] = n_scales
config["n_angles"] = n_angles
config["n_colors"] = n_colors
config["n_positions"] = n_positions
config["img_size"] = img_size
config["n_train"] = n_train
config["n_valid"] = n_valid
config["subset"] = generate_subset
config["target_bbox"] = target_bbox
config["target_segm"] = target_segm
config["min_mask_fill_rate"] = min_mask_fill_rate
config["max_mask_fill_rate"] = max_mask_fill_rate

pprint.pprint(config)

### Save or Send config

In [ ]:
p_data = Path("data")
p_config_tmp = p_data / "config.pkl"
with open(p_config_tmp, "wb") as file:
    pkl.dump(config, file)